In [ ]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import uuid
import mlflow
import mlflow.sklearn

from datetime import datetime

from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, r2_score, root_mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
# Directory config to project root to insure consistency across environments for project specific imports
from pyprojroot import here
os.chdir(here())

# Project specific imports
from ayne.utils.io import (
    save_dataframe,
    load_dataframe,
    save_artifacts,
    load_artifacts,
    save_processed_data,
    load_processed_data
)
from ayne.ml.models.serialize import (
    save_model,
    load_model,
    save_pipeline,
    load_pipeline
)

## Data Loading

I have set up multiple versions of my dataset, and the section below is used to swap between them for experimentation.

The main purpose behind this setup is to run experiments on data that has been imputed with and without the target included in the inputs for the imputation. I also investigate the effect of splitting the data into train and test sets before imputing.

In [ ]:
target = 'revenue'

### Imputed Data with Target Included

This version of the dataset uses all available features during imputation, including the target variable. It is primarily intended for exploratory analysis to understand the impact of including the target variable in the imputation process.

In [ ]:
# # Load dataset using modern Parquet format
# data = load_processed_data("imputed_data_full.parquet")

# # Data filters
# data = data.dropna(subset=['revenue'])
# # data = data[data["budget_missing"] == 0]
# # data = data[data["revenue_missing"] == 0]
# # data = data[data["budget_missing"] == 0 & data["revenue_missing"] == 0]
# # data = data[(data["budget_missing"] == 0) & (data["revenue_missing"] == 0)]

# # Split X/y
# # X = data.drop(columns=[target])
# X = data.drop(columns=[target, 'revenue_missing'])
# y = data[target]

# # Train/Test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.info()

### Imputed Data with Target Excluded

The target variable was excluded from the imputation process to prevent data leakage and ensure a more reliable model evaluation.  

Since missing values in the target column were not imputed, they will have to be dropped, reducing the available data for training. This approach prioritizes model integrity over dataset size, avoiding potential bias introduced by artificially inferred target values.


In [ ]:
# # Load dataset using modern Parquet format
# data = load_processed_data("imputed_data_no_revenue.parquet")

# # Data filters
# data = data.dropna(subset=['revenue'])
# # data = data[data["budget_missing"] == 0]
# # data = data[data["revenue_missing"] == 0]
# # data = data[data["budget_missing"] == 0 & data["revenue_missing"] == 0]
# # data = data[(data["budget_missing"] == 0) & (data["revenue_missing"] == 0)]

# # Split X/y
# X = data.drop(columns=[target])
# y = data[target]

# # Train/Test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.info()

### Imputed Data with Pre-Split Train/Test Sets

The dataset was first divided into training and testing subsets before performing imputation. 

This approach ensures that the imputation model is fitted solely on the training data, thereby preventing data leakage and preserving the integrity of model evaluation. Splitting the data before imputation also helps simulate a real-world scenario, ensuring that the imputed values are more robust and generalizable when applied to unseen data.


In [ ]:
# Load pre-split and imputed training/test datasets
# Using modern Parquet format with load_artifacts utility
X_train = load_artifacts("X_train_imputed.parquet")
X_test = load_artifacts("X_test_imputed.parquet")
y_train = load_artifacts("y_train.parquet")
y_test = load_artifacts("y_test.parquet")

X_train.info()

## MultiLabel Binarizer

Custom transformer to handle multi-label categorical features by encoding them into separate binary columns.  

This transformer:
- Splits values in the specified column using a delimiter (default: ",").
- Replaces spaces with underscores to ensure consistent naming.
- Applies `MultiLabelBinarizer` to encode the unique values as binary features.
- Preserves feature names with an optional prefix for clarity.
  
Designed for use within a `ColumnTransformer`, allowing seamless transformation of multiple multi-label features as part of a `Pipeline`.


In [ ]:
class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    """
    A transformer that applies MultiLabelBinarizer to a specified column.
    It splits the column values using a delimiter, replaces spaces with underscores,
    fits a MultiLabelBinarizer during fit, and transforms new data using the learned classes.
    
    Parameters
    ----------
    column : str
        The name of the column to encode.
    delimiter : str, default=","
        The delimiter used in the column to separate values.
    prefix : str, optional
        The prefix for the dummy columns. If None, uses the column name.
    """
    def __init__(self, column, delimiter=",", prefix=None):
        self.column = column
        self.delimiter = delimiter
        self.prefix = prefix if prefix is not None else column
        self.mlb_ = None
        
    def fit(self, X, y=None):
        X = X.copy()
        # Convert the column values into lists
        list_col = X[self.column].apply(
            lambda x: [i.strip().replace(' ', '_') for i in x.split(self.delimiter)] if pd.notnull(x) else []
        )
        self.mlb_ = MultiLabelBinarizer()
        self.mlb_.fit(list_col)
        return self
    
    def transform(self, X):
        X = X.copy()
        # Convert column values into lists as in fit
        list_col = X[self.column].apply(
            lambda x: [i.strip().replace(' ', '_') for i in x.split(self.delimiter)] if pd.notnull(x) else []
        )
        dummies = pd.DataFrame(
            self.mlb_.transform(list_col),
            columns=[f"{self.prefix}_{cls}" for cls in self.mlb_.classes_],
            index=X.index
        )
        # Drop the original column and add the dummies.
        X = X.drop(columns=[self.column])
        X = pd.concat([X, dummies], axis=1)
        return X
    
    def get_feature_names_out(self, input_features=None):
        return [f"{self.prefix}_{cls}" for cls in self.mlb_.classes_]

# To apply this transformer for multiple columns in a ColumnTransformer:
from sklearn.compose import ColumnTransformer

mlb_features = ['genre_names', 'production_country_name', 'spoken_languages']

transformers = [
    (f"mlb_{feat}", MultiLabelBinarizerTransformer(column=feat, delimiter=","), [feat])
    for feat in mlb_features
]

mlb = ColumnTransformer(
    transformers=transformers,
    remainder='passthrough',
    verbose_feature_names_out=False
)

# Set output to pandas dataframe
mlb.set_output(transform='pandas')

# Apply transformation
# transformed_data = ct.fit_transform(data)
# if not isinstance(transformed_data, pd.DataFrame):
#     transformed_data = pd.DataFrame(transformed_data)
    
# print(transformed_data.head())

In [ ]:
# Set up list of numerical coluns
num_cols = make_column_selector(dtype_include=['number'])
num_cols = num_cols(X_train)

# Display list
# for col in num_cols:
#     print(col)

print(num_cols)

In [ ]:
# Set up list of categorical coluns
cat_cols = make_column_selector(dtype_include=['object'])
cat_cols = cat_cols(X_train)

# Display list
# for col in cat_cols:
#     print(col)

print(cat_cols)

## Scalers and Encoders  

ColumnTransformer pipeline to preprocess numerical and categorical features for modeling.  

This transformation includes:  
- **StandardScaler**: Normalizes numerical features to ensure consistent scale.  
- **OneHotEncoder**: Encodes categorical variables as binary columns while ignoring unknown categories.  
- **OrdinalEncoder**: Assigns ordered integer values to categorical features, with unknown values handled explicitly.  

The transformer is configured to output a pandas DataFrame for seamless integration into the modeling pipeline.  


In [ ]:
modeling_transformer = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), [
            'metascore', 'rotten_tomatoes_rating', 'meta_critic_rating', 'budget', 'tmdb_vote_count', 'tmdb_vote_average', 'runtime_in_min', 
            'tmdb_popularity', 'imdb_rating', 'imdb_votes', 'total_wins', 'total_noms', 'oscar_wins', 'oscar_noms', 'bafta_wins', 'bafta_noms', 
            'release_year', 'release_month', 'release_day', 'movie_age', 
            # 'roi', 'revenue'
            ]),
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['age_rating']),
        ('OrdinalEncoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=999999), ['first_billing_actor', 'second_billing_actor', 'third_billing_actor', 'main_director', 'main_writer', 'secondary_writer'])
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False,
)

# Set output to pandas dataframe
modeling_transformer.set_output(transform='pandas')

# transformed_data = modeling_transformer.fit_transform(data)
# transformed_data.head()

In [ ]:
# Define a function transformer to drop unwanted columns.
def drop_unwanted_columns(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    return df.drop(columns=columns, errors='ignore')

# List of unwanted columns
columns_to_drop = ['production_company_name', 'director', 'writer', 'actors', 'title', 'release_date']

# initialize dropper
dropper = FunctionTransformer(drop_unwanted_columns, kw_args={'columns': columns_to_drop})

## Applying transformation

### Setting up the pipeline

This pipeline includes:  
- **modeling_transformer**: Applies scaling and encoding transformations.  
- **mlb**: Binarizes multi-label categorical features.  
- **dropper**: Removes unnecessary columns for cleaner data.  

The transformed dataset is ready for model training and evaluation.  

In [ ]:
# Build pipeline
scaling_pipeline = Pipeline(steps=[
    ('modeling_transformer', modeling_transformer),
    # ('mlb', mlb_transformer),
    ('mlb', mlb),
    ('dropper', dropper)
])

# scaled_data = scaling_pipeline.fit_transform(data)
# scaled_data.info()

In [ ]:
# Fit the pipeline on the training data
X_train_scaled = scaling_pipeline.fit_transform(X_train, y_train)

# Transform the test data using the same pipeline
X_test_scaled = scaling_pipeline.transform(X_test)

In [ ]:
X_train_scaled.head()

## Model Training and Optimization

### Evaluation Function

The `model_evaluation` function assesses multiple regression models using 5-fold cross-validation.  
It computes and reports the mean R² and RMSE scores for each model, providing a comparative analysis of their performance.  

- **R² Score**: Measures how well the model explains variance in the target variable.  
- **RMSE Score**: Indicates the model's prediction error in the original units of the target variable.  

The results are stored in a dictionary and printed in a structured format for easy interpretation.  


In [ ]:
def model_evaluation(models: dict, X_train, y_train, w_train=None):
    """
    Evaluates regression models using Cross Validation.
    Returns a report with R² and RMSE scores.
    """
    
    evaluation_report = {}

    for model_name, model in models.items():
        # R² Score
        r2_scores = cross_val_score(
            model, 
            X_train, 
            y_train, 
            cv=5, 
            scoring="r2", 
            n_jobs=-1
        )
        
        # RMSE Score
        rmse_scores = cross_val_score(
            model, X_train, y_train, 
            cv=5, 
            scoring=make_scorer(lambda y, y_pred: root_mean_squared_error(y, y_pred)), 
            n_jobs=-1
        )

        # Store results
        evaluation_report[model_name] = {
            "R² Mean": np.mean(r2_scores),
            "RMSE Mean": np.mean(rmse_scores)
        }
        
        # Print results
        print(f"{model_name}:")
        print(f"- CV R² scores: {' | '.join([f'{r * 100:.1f}%' for r in r2_scores])}")
        print(f"- Mean R²: {np.mean(r2_scores) * 100:.1f}%")
        print("- -----------------------------------------------------------------------------------------------------------")
        print(f"- CV RMSE scores: {' | '.join([f'{r:,.0f}$' for r in rmse_scores])}")
        print(f"- Mean RMSE: {np.mean(rmse_scores):,.0f}$")
        print()


    
    return evaluation_report

models = {
    # "Linear Regression": LinearRegression(),
    # "Ridge Regression": Ridge(alpha=1.0),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=50, random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGBoost Regressor": XGBRegressor(random_state=42),
    "LightGBM Regressor": LGBMRegressor(random_state=42),
    # "KNeighbors Regressor": KNeighborsRegressor(),
    # "SVR": SVR()
}


In [ ]:
# Run initial evaluation
_ = model_evaluation(models, X_train_scaled, y_train)

### Parameter Tuning Function

The `tune_params_regression` function optimizes hyperparameters for a regression model using `GridSearchCV`.  
It performs 5-fold cross-validation to identify the best parameter set based on a specified scoring metric.

- **MLflow Integration**: Logs model parameters, performance metrics (R², RMSE), and training data artifacts.  
- **Experiment Tracking**: Assigns unique experiment and run names for reproducibility.  
- **Artifact Storage**: Saves datasets and trained models for further analysis.  
- **Performance Reporting**: Displays the best hyperparameters and test set evaluation results.  


In [ ]:
def tune_params_regression(model, X_train, X_test, y_train, y_test, param_grid, scoring='r2'):
    """
    Evaluates a regression model using cross-validation and logs parameters and metrics with MLflow.
    Returns the best estimator.
    
    Parameters
    ----------
    model : estimator
        The regression model instance.
    X_train, X_test : DataFrame or array-like
        Training and testing features.
    y_train, y_test : array-like
        Training and testing target values.
    param_grid : dict
        The grid of parameters for GridSearchCV.
    scoring : str, default='r2'
        The scoring metric for GridSearchCV.
        
    Returns
    -------
    best_model : estimator
        The best estimator found by GridSearchCV.
    """
    # Define an experiment name using model type and timestamp.
    experiment_name = f"Exp_{model.__class__.__name__}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
    mlflow.set_experiment(experiment_name)

    # Define a custom run name and run_id.
    run_name = f"{model.__class__.__name__}_Run_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
    run_id = f"{model.__class__.__name__}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_{str(uuid.uuid4())[:8]}"

    with mlflow.start_run(run_name=run_name):

        # MlFlow logs
        mlflow.log_param("model_type", model.__class__.__name__)

        # Save training datasets using modern utilities
        save_artifacts(X_train, "X_train", format="parquet")
        save_artifacts(pd.DataFrame(y_train), "y_train", format="parquet")

        # Log these files as artifacts to MLflow
        mlflow.log_artifact("data/artifacts/X_train.parquet", artifact_path="datasets")
        mlflow.log_artifact("data/artifacts/y_train.parquet", artifact_path="datasets")
        
        # Initialize GridSearchCV for regression.
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring=scoring,
            cv=5,
            n_jobs=-1
        )
        
        # Fit the model on the training data.
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        # Predict on the test set.
        y_pred = best_model.predict(X_test)
        
        # Compute R² and RMSE.
        r2 = r2_score(y_test, y_pred)
        rmse = root_mean_squared_error(y_test, y_pred)
        
        # Log metrics to MLflow.
        mlflow.log_metric("r2_score", r2)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_params(grid_search.best_params_)
        mlflow.sklearn.log_model(best_model, f"Tuned_{model.__class__.__name__}", input_example=X_train.iloc[:20])
        
        # Save model using modern joblib-based approach
        model_metadata = {
            "model_type": model.__class__.__name__,
            "r2_score": r2,
            "rmse": rmse,
            "best_params": grid_search.best_params_,
            "cv_folds": 5,
            "scoring": scoring,
            "n_features": X_train.shape[1],
            "feature_names": X_train.columns.tolist() if hasattr(X_train, 'columns') else None
        }
        model_filename = f"{model.__class__.__name__}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        save_model(best_model, model_filename, metadata=model_metadata)
        
        # Print out the results.
        r2_formatted = f"{r2 * 100:.1f}%"
        rmse_formatted = f"{rmse:,.0f}$"
        print("----------------------------------------")
        print(f"Test Data Results")
        print(f"- R² Score: {r2_formatted}")
        print(f"- RMSE: {rmse_formatted}")
        print("----------------------------------------")
        print("Best Parameters:")
        for param, value in grid_search.best_params_.items():
            print(f"- {param}: {value}")
        print("----------------------------------------")
        print(f"Model saved to: data/artifacts/models/{model_filename}.joblib")
        
        return best_model


### Evaluating Model Performance

#### Evaluation Function

This function, `evaluate_model`, takes a trained regression model along with its test dataset (features and target values) and computes essential performance metrics. It calculates the R² score (converted to a percentage) and the Root Mean Squared Error (RMSE), then prints these results along with the model's name. Additionally, it visualizes the distribution of residuals via a histogram to help assess model errors. Finally, it returns a dictionary containing the R² and RMSE scores for further analysis.


In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluates a trained regression model on test data.
    
    Parameters
    ----------
    model_name : str
        The name of the model for display.
    model : estimator
        The trained regression model.
    X_test : DataFrame or array-like
        Test features.
    y_test : array-like
        True target values.
    
    Returns
    -------
    dict
        A dictionary containing R² and RMSE scores.
    """
    # Generate predictions
    y_pred = model.predict(X_test)
    
    # Compute evaluation metrics
    r2 = r2_score(y_test, y_pred) * 100  # Convert to percentage
    rmse = root_mean_squared_error(y_test, y_pred)  # Root Mean Squared Error

    # Get model name
    model_name = model.__class__.__name__
    
    # Print results
    print(f"{model_name}:")
    print(f"  - R² Score: {r2:.1f}%")
    print(f"  - RMSE: {rmse:,.0f}$\n")

    # Convert to 1D if necessary
    y_test = y_test.values.flatten()

    # Plot Residuals
    residuals = y_test - y_pred
    plt.figure(figsize=(8, 5))
    sns.histplot(residuals, kde=True, bins=30)
    plt.xlabel("Residuals")
    plt.ylabel("Density")
    plt.title(f"Residual Distribution for {model_name}")
    plt.show()
    
    return {"R²": r2, "RMSE": rmse}

#### Feature Importances function

This function calculates and displays the feature importances of a trained model. It extracts the feature importance values from the model, sorts them in descending order, and plots the top 20 most important features.

In [ ]:
def display_feature_importances(model, dataframe):
    """
    Displays and plots the importance of features for a trained model.
    
    This function extracts the feature importances from the trained model, 
    sorts them, and visualizes the top 20 most important features using a 
    horizontal bar chart.
    """
    # Create a DataFrame with feature names and their corresponding importance scores
    importance_df = pd.DataFrame({
        'Feature': dataframe.columns,           # Feature names
        'Importance': model.feature_importances_  # Feature importance values from the model
    })

    # Sort features by importance in descending order
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # Plot the top 20 most important features
    plt.figure(figsize=(10, 6))  # Set the plot size
    plt.barh(importance_df['Feature'][:20], importance_df['Importance'][:20], color='skyblue')  # Horizontal bar chart for top 20 features
    plt.xlabel('Importance')  # Label the x-axis
    plt.title(f'Top Feature Importances from {type(model).__name__}')  # Set the plot title with model type
    plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
    plt.show()  # Display the plot


### Random Forrest

In [ ]:
# param_grid_rf = {
#     'n_estimators': [100, 300, 500],  
#     'max_depth': [None, 10, 20, 30],  
#     'min_samples_split': [2, 5, 10],  
#     'min_samples_leaf': [1, 2, 4],  
#     'max_features': ['auto', 'sqrt', 'log2']  
# }

# rf = RandomForestRegressor(random_state=42)

# # Example: Tuning a Random Forest Regressor
# best_rf = tune_params_regression(
#     model=rf,
#     X_train=X_train_scaled, 
#     X_test=X_test_scaled,
#     y_train=y_train, 
#     y_test=y_test,
#     param_grid=param_grid_rf,
#     scoring='r2'
# )


### Gradient Boosting Regressor

In [ ]:
# Init a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)

# Define parameter grid
param_grid_gbr = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0]
}

# Tune model
# best_gbr = tune_params_regression(
#     gbr,
#     X_train_scaled, 
#     X_test_scaled,
#     y_train, 
#     y_test,
#     param_grid_gbr
# )


### LightGBM Regressor

In [ ]:
# Init a XGBoost Regressor
lgbm = LGBMRegressor(random_state=42)

# Set parameter grid
param_grid_lgbm = {
    'n_estimators': [200, 300, 400],
    'max_depth': [5, 7, 10, -1],
    'learning_rate': [0.05, 0.1, 0.15],
    'num_leaves': [5, 10, 30],
    'subsample': [0.8, 1.0]
}

# Tune model
best_lgbm = tune_params_regression(
    lgbm,
    X_train_scaled, 
    X_test_scaled,
    y_train, 
    y_test,
    param_grid_lgbm
)

In [ ]:
_ = evaluate_model(best_lgbm, X_test_scaled, y_test)


In [ ]:
display_feature_importances(best_lgbm, X_train_scaled)

In [ ]:
# Example: Loading a saved model with metadata
# 
# # Load model and metadata together
# model, metadata = load_model("data/artifacts/models/LGBMRegressor_20241122_143000.joblib", load_metadata=True)
# 
# # View model information
# print("Model Information:")
# print(f"- Type: {metadata.get('model_class')}")
# print(f"- R² Score: {metadata.get('r2_score', 0) * 100:.1f}%")
# print(f"- RMSE: ${metadata.get('rmse', 0):,.0f}")
# print(f"- Saved at: {metadata.get('saved_at')}")
# print(f"- Features: {metadata.get('n_features')}")
# 
# # Use the model for predictions
# # predictions = model.predict(X_new)